生成数据集

In [32]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2,-3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

读取数据集
调用框架中现有的API来读取数据。 我们将features和labels作为API的参数传递，并通过数据迭代器指定batch_size。 此外，布尔值is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据。

In [33]:
# 构造一个pytorch的数据迭代器
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

使用iter构造Python迭代器，并使用next从迭代器中获取第一项

In [34]:
next(iter(data_iter))

[tensor([[-3.7562e-01,  5.5778e-01],
         [-1.1200e+00, -1.1064e+00],
         [-1.4860e+00, -3.1920e+00],
         [ 1.5494e+00,  1.0962e+00],
         [ 2.4784e-04,  4.2248e-01],
         [ 1.7277e-01, -5.4033e-01],
         [ 1.4968e-01, -1.8218e-02],
         [-8.5948e-01,  8.7561e-01],
         [-1.0421e+00, -1.0107e+00],
         [ 6.5947e-01,  4.4029e-01]]),
 tensor([[ 1.5689],
         [ 5.7116],
         [12.0703],
         [ 3.5668],
         [ 2.7750],
         [ 6.3884],
         [ 4.5682],
         [-0.4946],
         [ 5.5611],
         [ 4.0229]])]

我们首先定义一个模型变量net，它是一个Sequential类的实例。 Sequential类将多个层串联在一起。
当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推。
在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential。 但是由于以后几乎所有的模型都是多层的，在这里使用Sequential会让你熟悉“标准的流水线”。


在PyTorch中，全连接层在Linear类中定义。 值得注意的是，我们将两个参数传递到nn.Linear中。 第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1。

In [35]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2,1))

在使用net之前，我们需要初始化模型参数。如在线性回归模型中的权重和偏置。
深度学习框架通常有预定义的方法来初始化模型的参数。
在这里，我们指定每个权重参数应该从均值为0，方差为0.01的正态分布中随机采样，偏置参数将初始化为零。

正如我们在构造nn.Linear时指定输入和输出尺寸一样， 现在我们能直接访问参数以设定它们的初始值。 我们通过net[0]选择网络中的第一个图层， 然后使用weight.data和bias.data方法访问参数。 我们还可以使用替换方法normal_和fill_来重写参数值。

In [36]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

定义损失函数
计算均方平均误差使用的是MSLoss类，也称为平方L2范数。
默认情况下，它返回所有样本损失的平均值。

In [37]:
loss = nn.MSELoss()

定义优化器
小批量随机梯度下降算法是一种优化神经网络的标准工具， PyTorch在optim模块中实现了该算法的许多变种。
当我们实例化一个SGD实例时，我们要指定优化的参数 （可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。 小批量随机梯度下降只需要设置lr值，这里设置为0.03。

In [38]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练
通过pytorch框架我们只需要较少的代码
回顾一下：在每个迭代周期里，我们将完整遍历一次数据集（train_data）， 不停地从中获取一个小批量的输入和相应的标签。 对于每一个小批量，我们会进行以下步骤:

通过调用net(X)生成预测并计算损失l（前向传播）。

通过进行反向传播来计算梯度。

通过调用优化器来更新模型参数。

为了更好的衡量训练效果，我们计算每个迭代周期后的损失，并打印它来监控训练过程。

In [39]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch :{epoch + 1}, loss :{l:f}')

epoch :1, loss :0.000188
epoch :2, loss :0.000104
epoch :3, loss :0.000105


下面我们通过比较生成数据集的真实参数和通过有限数据训练获得的模型参数。要访问参数，我们首先从net访问所需的层，然后读取该层的权重和偏置。正如在从零开始实现中一样，我们估计得到的参数与生成数据的真实参数非常接近。

In [40]:
w = net[0].weight.data
print('权重的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

权重的估计误差： tensor([ 8.5235e-05, -9.7704e-04])
b的估计误差： tensor([5.8174e-05])


总结
我们可以使用PyTorch的高级API更简洁地实现模型。

在PyTorch中，data模块提供了数据处理工具，nn模块定义了大量的神经网络层和常见损失函数。

我们可以通过_结尾的方法将参数替换，从而初始化参数。